In [1]:
from bertviz import head_view
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import random
import re
import os
from sklearn.metrics import classification_report

In [2]:
def get_questions(path):
    '''
        Read in all the questions with their ids from a specific path
    '''

    questions = {}
    for root, dirs, files in os.walk(path):
        for file in files:
            if file=='context.txt':
                with open(os.path.join(root,file)) as f:
                    q_full = f.read().split('<br>')
                    q_text = q_full[2]
                    q_title = q_full[0]
                    q_context = ' '.join([x for x in re.split("[?.!;]", q_text) if x!=""][-3:])
                    # Strip off newline and tab characters
                    q_context = q_context.replace('\n', '').replace('\t', '')
                    q_context = q_context + " " + q_title
                    questions[root.split('/')[-1]] = q_context

    return questions

In [3]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [4]:
# Set random seeds for reproducibility on a specific machine
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.manual_seed(1)
torch.cuda.manual_seed(1)
random.seed(1)
np.random.seed(1)
np.random.RandomState(1)

RandomState(MT19937) at 0x1A480C3678

In [5]:
pd.set_option('display.max_colwidth', None)

data = 'askparents'

pred_file = '../../preds/classifier_askparents_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:45_multigpu:True_labels:ds_frac:1.csv'

dev = pd.read_csv(pred_file, header=0, sep='\t')
dev_sentences = dev['Sentence'].tolist()
dev_labels_DS = dev['DS_Label'].values
dev_labels_Maj = dev['Majority_label'].values
dev['Pred_Label'] = dev['Pred_Label'].astype(int)
dev_preds = dev['Pred_Label'].values
dev = dev.set_index('ID')
questions = get_questions('../../rawdata/' + data)
dev['Question'] = dev.index.map(lambda x:questions[x.split('-')[0]])
dev

,Majority_label,DS_Label,Sentence,Post.ID,Reply.ID,Sent.Num,Pred_Label,Question
ID,,,,,,,,
bizagh-4-1,0,0,Did n’t screw me up any except maybe having a sense of humor that was a little too advanced for my age,bizagh,4.0,1,0,I feel like I need to do damage control Her grandparents think I’m over reacting but I honestly don’t give a shit I feel like I failed as a parent So my three year old watched something awful!
98xghs-4-2,0,0,"There 's nothing to argue about here , but she has taken the liberty upon herself to express strong feelings YOU WEREN'T ASKING FOR , and you need her to justify them .",98xghs,4.0,2,0,"The end Rant over, thank you all Why is my Mother so negative over my new job?"
auxnv5-2-2,0,0,Do they behave like this with their parents ?,auxnv5,2.0,2,0,"I‘m starting to dread babysitting because I’m tired of their behaviour, and I find their negative comments draining and irritating I talked to their mother (my sister) and she has said she will talk to them about it, but that was weeks ago, and their rude and mean comments persist Parents, how do you deal with kids who seem to find a lot of joy in making mean and rude comments How to discourage mean & rude comments?"
dri3p0-2-7,0,0,"If they can abide by that , great .",dri3p0,2.0,7,0,"I'm not sure what route I should take I'm debating on letting my husband take our son and meet them in public so they can visit or just dropping them completely from our lives I however, will have nothing to do with them anymore Awful grandparent behavior"
c823ha-6.1-4,0,0,Ibuprofen reduces the associated inflammation which Tylenol wo n’t .,c823ha,6.1,4,0,"I have a ten month old who is a good baby and hardly ever cry’s but ere lately it’s all he’s done I’ve tried everything I’m about to lose my sanity if this doesn’t stop, what can I do Why will my 10 month old not stop crying?"
...,...,...,...,...,...,...,...,...
dri3p0-2-5,0,0,Full stop .,dri3p0,2.0,5,0,"I'm not sure what route I should take I'm debating on letting my husband take our son and meet them in public so they can visit or just dropping them completely from our lives I however, will have nothing to do with them anymore Awful grandparent behavior"
98xghs-1-3,1,1,You do n't actually have to tell her anything of any substance .,98xghs,1.0,3,1,"The end Rant over, thank you all Why is my Mother so negative over my new job?"
7kx59j-1-3,0,0,https://imgur.com/a/hm35yEdit : thanks for the gold whoever sent it !,7kx59j,1.0,3,1,"If anybody could help me find a 2012, striped baby doll rattle plush, from Cabbage Patch, with blue eyes, it would make my niece SO happy Pictures of the doll here: https://imgur com/gallery/HycFW Help me help my niece out"


In [6]:
dev[(dev['DS_Label']==1)& (dev['Pred_Label']==1)].sample(50)

,Majority_label,DS_Label,Sentence,Post.ID,Reply.ID,Sent.Num,Pred_Label,Question
ID,,,,,,,,
54y27a-3-2,1,1,"I would encourage finding a therapist to add to your "" support team "" .",54y27a,3.0,2,1,Any tips or tricks Are you glad you decided to become a parent Thank you Parenting with a History of Depression?
be9axi-1-4,0,1,When he ’s older if he requests to watch something like that I might let him but I ’m not going to encourage it or introduce it either .,be9axi,1.0,4,1,TL DR: What’s your rule for TV content Why do you do it this way Violent TV shows and dramas
bizagh-1.1-1,1,1,Try not to be too hard on yourself .,bizagh,1.1,1,1,I feel like I need to do damage control Her grandparents think I’m over reacting but I honestly don’t give a shit I feel like I failed as a parent So my three year old watched something awful!
7hzot3-1-0,1,1,Check out ahaparenting.com if you have n’t already .,7hzot3,1.0,0,1,"Putting things in his mouth, climbing the chair or table Any tips Thank you in advance A kid that's too much to handle. Help."
48zlnr-2-0,1,1,My biggest advice is bring in healthy food that s all pretty and cartoony ( pinterest anyone ) .,48zlnr,2.0,0,1,"Because to me, that does not seem like a 3 year-old who's developing healthy eating habits I know there's really anything I can do about it because it's not my place to do so, but I guess I'm just venting I feel like I'm watching a car wreck in super slow motion ""No candy until you finish your pizza..."" That's bad, right?"
duvs75-4-0,1,1,Play with them - kids ' imaginations are beautiful and a great source of fun .,duvs75,4.0,0,1,I imagine there are plenty of weekends with no major timesink activities Like what do you do with a toddler for that long Edit: assuming there is only 1kid and no siblings What exactly do you do with a toddler all weekend?
54y27a-1-0,1,1,"Make sure you have a strong , committed , supportive partner .",54y27a,1.0,0,1,Any tips or tricks Are you glad you decided to become a parent Thank you Parenting with a History of Depression?
duvs75-8-2,1,1,Do n't overthink it and do n't try to cram too much in .,duvs75,8.0,2,1,I imagine there are plenty of weekends with no major timesink activities Like what do you do with a toddler for that long Edit: assuming there is only 1kid and no siblings What exactly do you do with a toddler all weekend?
dgpwtd-2.1-2,1,1,"If a kids hitting you , you have every right to push them away .",dgpwtd,2.1,2,1,Where to draw the line What is reasonable in that situation Should I have picked him up and sat him in the next room like his mom did How much force is reasonable to use on a 4 y/o?


In [7]:
dev[(dev['DS_Label']==1) & (dev['Pred_Label']==0)].sample(50)

,Majority_label,DS_Label,Sentence,Post.ID,Reply.ID,Sent.Num,Pred_Label,Question
ID,,,,,,,,
9yhugm-2-2,1,1,"* ) , I 'd go up to him afterwards and nicely mention that you did n't find that remark at all helpful and you 'd appreciate it if he would n't repeat it to you again .",9yhugm,2.0,2,0,I don’t know what to do I think his behavior is unacceptable What do you guys think My [15M] Biology teacher [42M] is disrespectful.
be9axi-5-0,1,1,"We do nt watch scary movies or things with monsters they do nt understand that its not real , but we watch greys anatomy together my 5 year old daughter likes it",be9axi,5.0,0,0,TL DR: What’s your rule for TV content Why do you do it this way Violent TV shows and dramas
71r6gd-3-0,1,1,"At 6 , he should know where his food comes from .",71r6gd,3.0,0,0,[deleted] How to explain to a kid the reality of going hunting
cz58ee-3-0,1,1,In addition to warning Mr.,cz58ee,3.0,0,0,"I can't control how the kids react, but I understand how it would make the other grandparent feel I hope this isn't confusing Thanks for your time When one grandparent ALWAYS shows up and the other only occasionally, how do you deal with the fallout?"
4pn5x8-5-2,1,1,Would I do it ?,4pn5x8,5.0,2,0,"I witnessed this today and just looking for other parents opinion/experience on it What age do you think it is appropriate to leave a child in a car not running with the windows down in a private daycare parking lot, the daycare is locked to people coming in and you have no visual of the car or child while you are in the building Age of child left in car"
dgpwtd-2-1,1,1,"Unless it 's a safety issue , I do n't put my hands on my kids in a moment like that - it sends the message that I 'm allowed to touch their body without consent .",dgpwtd,2.0,1,0,Where to draw the line What is reasonable in that situation Should I have picked him up and sat him in the next room like his mom did How much force is reasonable to use on a 4 y/o?
7hzot3-3-2,0,1,"If he lashes out by yelling , then gets spanked for it , that only teaches him physical violence is the answer , which does n’t progress anything in a positive way .",7hzot3,3.0,2,0,"Putting things in his mouth, climbing the chair or table Any tips Thank you in advance A kid that's too much to handle. Help."
c4so1m-4-0,1,1,Generally talking about her like she 's not right there .,c4so1m,4.0,0,0,"I decided it would be best to go visit them at Granny's and take them places from there instead of bringing them to my house during my time with them This caused issues with my roommate needing a different sitter for her daughter while I was with my kids, and I felt bad for the girl and my daughter because they both enjoyed when they got to play together I also felt bad that I didn't get to help her with her homework very often anymore, but my kids come first What’s the most disrespectful thing someone has done to your kids in front of you?"
c4so1m-6-4,1,1,"All three of our boys were there and when I answered ( with a huge smile ) “ a boy ” she looked at the kids and said , “ Oh no , I ’m sorry .",c4so1m,6.0,4,0,"I decided it would be best to go visit them at Granny's and take them places from there instead of bringing them to my house during my time with them This caused issues with my roommate needing a different sitter for her daughter while I was with my kids, and I felt bad for the girl and my daughter because they both enjoyed when they got to play together I also felt bad that I didn't get to help her with her homework very often anymore, but my kids come first What’s the most disrespectful thing someone has done to your kids in front of you?"


In [8]:
dev[(dev['DS_Label']==0) & (dev['Pred_Label']==1)].sample(50)

,Majority_label,DS_Label,Sentence,Post.ID,Reply.ID,Sent.Num,Pred_Label,Question
ID,,,,,,,,
8kic2y-1-2,0,0,"It 's not just about playing with him , you have a huge resposibility , looking after a child",8kic2y,1.0,2,1,[deleted] How much pay would be appropriate for this babysitting job?
7hzot3-1-1,0,0,I think that would be a good start in taking a new direction in your reading and research on child development .,7hzot3,1.0,1,1,"Putting things in his mouth, climbing the chair or table Any tips Thank you in advance A kid that's too much to handle. Help."
71r6gd-3-1,0,0,"Burgers come from cows , chicken nuggets come from chicken , bacon comes from pigs etc .",71r6gd,3.0,1,1,[deleted] How to explain to a kid the reality of going hunting
4xv4lf-1-1,0,0,"Like you said , you do n't want to reject their feelings .",4xv4lf,1.0,1,1,[deleted] Daycare kids calling me dad
99o434-4-0,0,0,I just buy stuff on clearance racks and check local facebook swap groups .,99o434,4.0,0,1,[deleted] Kids Clothing
ackme2-1.1-1,0,0,"Both of you must take your bounds seriously , otherwise there is no point in trying .",ackme2,1.1,1,1,Or am I in the wrong here Thanks a heap mums and dads <3 Feeling overly protected. Am I in the wrong here?
54y27a-3-1,0,0,"Dr , husband , friends , and extended family .",54y27a,3.0,1,1,Any tips or tricks Are you glad you decided to become a parent Thank you Parenting with a History of Depression?
a8v90n-5-2,0,0,He needs his own space .,a8v90n,5.0,2,1,"In this case all three might be ""guilty"" however I would like to know if it is possible for the breastfed child to sleep through whole night and what methods are there to help him with it (ideally which not include extensive crying) So how do you make the Sandman bring you good dreams :) Can a 11 month old breastfed child sleeping together with parents in bed sleep whole night?"
droq6l-2-1,0,0,"for when we will be gone awhile , but usually I just use one of those fold up changing pads that has pockets for diapers and wipes , and diaper rash cream .",droq6l,2.0,1,1,What's in your diaper bag?


## Model loading

In [9]:
saved_model = '../../saved_pretrained/classifier_askparents_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:45_multigpu:True_labels:ds_frac:1/'

tokenizer = AutoTokenizer.from_pretrained(saved_model)
model = AutoModelForSequenceClassification.from_pretrained(saved_model)

model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [10]:
def predict(sentence):
    '''
        Returns classification value - 0,1 given sentence
    '''
    
    input_ids = torch.tensor(tokenizer.encode(sentence)).unsqueeze(0)
#     print(input_ids)
    with torch.no_grad():
        logits = model(input_ids)[0]
    print(logits)

In [11]:
print(dev.loc['7kxtia-3-0','Sentence'], "\n", 
      dev.loc['dudwxa-7-3','Sentence'], "\n",
      dev.loc['duvs75-4-0','Sentence'], "\n",
      dev.loc['dri3p0-4-0','Sentence'], "\n",
      dev.loc['54y27a-3-2','Sentence'], "\n",
      dev.loc['bizagh-1.1-1','Sentence'], "\n",
      dev.loc['54y27a-2-7','Sentence'], "\n",
      dev.loc['98xghs-1-3', 'Sentence'], "\n",
      dev.loc['c823ha-7-6', 'Sentence'])

If my parents hit me past 18 , I would call the cops . 
 Anything but cosleep . 
 Play with them - kids ' imaginations are beautiful and a great source of fun . 
 Cut them out of your life for the sake of your child . 
 I would encourage finding a therapist to add to your " support team " . 
 Try not to be too hard on yourself . 
 I talked on Reddit with others to get support and ideas . 
 You do n't actually have to tell her anything of any substance . 
 I suggest trying a dose of tylenol .


In [16]:
predict(dev.loc['54y27a-2-7', 'Sentence'])

tensor([[ 0.1893, -0.1189]])


## Attention Heads View

In [17]:
def show_head_view(model, tokenizer, sentence_a, sentence_b=None):
    inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True, return_token_type_ids=True)
    input_ids = inputs['input_ids']
    if sentence_b:
        token_type_ids = inputs['token_type_ids']
        with torch.no_grad():
            attention = model(input_ids, token_type_ids=token_type_ids)[-1]
        import ipdb; ipdb.set_trace()
        sentence_b_start = token_type_ids[0].tolist().index(1)
    else:
        with torch.no_grad():
            attention = model(input_ids)[-1]
        sentence_b_start = None
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)    
    head_view(attention, tokens, sentence_b_start)

In [19]:
sentence = dev.loc['54y27a-2-7']['Sentence']

In [20]:
show_head_view(model, tokenizer, sentence_a=sentence)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Personal narrative analysis 

In [26]:
dev_personal = dev[dev['Sentence'].str.contains(' i ') | dev['Sentence'].str.contains(' I ')]
dev_personal

,Majority_label,DS_Label,Sentence,Post.ID,Reply.ID,Sent.Num,Pred_Label,Question
ID,,,,,,,,
54y27a-4-0,0,0,I was 100 % certain I 'd suffer postnatal depression .,54y27a,4.0,0,0,Any tips or tricks Are you glad you decided to become a parent Thank you Parenting with a History of Depression?
4xv4lf-3-3,0,0,"Ideally , I think it would help everyone if there 's a lead - up to your departure , as opposed to them just showing up one day and you 're gone .",4xv4lf,3.0,3,0,[deleted] Daycare kids calling me dad
c4so1m-1-0,0,0,"If i 'm following this correctly , you have a roommate but your kids do n't live with you full time?I'm really appalled by your roommate taking her anger out on the children directly like that .",c4so1m,1.0,0,0,"I decided it would be best to go visit them at Granny's and take them places from there instead of bringing them to my house during my time with them This caused issues with my roommate needing a different sitter for her daughter while I was with my kids, and I felt bad for the girl and my daughter because they both enjoyed when they got to play together I also felt bad that I didn't get to help her with her homework very often anymore, but my kids come first What’s the most disrespectful thing someone has done to your kids in front of you?"
5l0ln3-2.1-0,0,1,Yup this is why I went straight to suggesting white noise etc .,5l0ln3,2.1,0,0,"Maybe bed time is none of my business But, how can I approach the parents and get them to quiet their kid at night How to confront neighbors about their lack of control"
cl69kp-1.1-0,0,0,"Haha , I do n't think so .",cl69kp,1.1,0,0,"I have two sets of twins and I'm expecting triplets soon I'm a full time stay at home mother and wasn't expecting our family to grow quite so quickly So I'd like some advice on raising a larger family, please Parents with 7+ kids, what are best tips to handle them?"
...,...,...,...,...,...,...,...,...
54y27a-2-4,0,1,I used formula ( meds required ) so my husband did feedings to ensure I got 8 + hours of sleep a night .,54y27a,2.0,4,0,Any tips or tricks Are you glad you decided to become a parent Thank you Parenting with a History of Depression?
droq6l-2-1,0,0,"for when we will be gone awhile , but usually I just use one of those fold up changing pads that has pockets for diapers and wipes , and diaper rash cream .",droq6l,2.0,1,1,What's in your diaper bag?
844azw-1-3,0,0,"When she would wait until the end of the routine and say she needed the bathroom again , I 'd just let her and then tuck her in with no fuss when she was done .",844azw,1.0,3,1,"I've actually thought maybe we should giver her MORE water before bed as that might satisfy her mental need to go She does not do this during the day at all, is not having accidents, and will use the potty by herself in any other situation It's only naps/nighttime Toddler saying she needs to go potty everytime she gets tucked into bed"


In [27]:
from sklearn.metrics import classification_report
print(classification_report(dev_personal['DS_Label'], dev_personal['Pred_Label']))

              precision    recall  f1-score   support

           0       0.76      0.89      0.82        76
           1       0.62      0.37      0.46        35

    accuracy                           0.73       111
   macro avg       0.69      0.63      0.64       111
weighted avg       0.71      0.73      0.71       111



In [29]:
from collections import Counter
print(Counter(dev_personal['Pred_Label']))
print(Counter(dev_personal['DS_Label']))

Counter({0: 90, 1: 21})
Counter({0: 76, 1: 35})


In [57]:
test2 = pd.read_csv('../../preds/test/classifier_askparents_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:2_multigpu:True_labels:ds_frac:1_TEST.csv', sep='\t', header=0)
test2['Sentence'] = test2['Sentence'].str.lower()
test2_personal = test2[test2['Sentence'].str.contains(' me ') | test2['Sentence'].str.contains(' my ') | test2['Sentence'].str.contains(' we ')]

test13 = pd.read_csv('../../preds/test/classifier_askparents_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:13_multigpu:True_labels:ds_frac:1_TEST.csv', sep='\t', header=0)
test13['Sentence'] = test13['Sentence'].str.lower()
test13_personal = test13[test13['Sentence'].str.contains(' me ') | test13['Sentence'].str.contains(' my ') | test13['Sentence'].str.contains(' we ')]

test20 = pd.read_csv('../../preds/test/classifier_askparents_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:20_multigpu:True_labels:ds_frac:1_TEST.csv', sep='\t', header=0)
test20['Sentence'] = test20['Sentence'].str.lower()
test20_personal = test20[test20['Sentence'].str.contains(' me ') | test20['Sentence'].str.contains(' my ') | test20['Sentence'].str.contains(' we ')]

test45 = pd.read_csv('../../preds/test/classifier_askparents_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:45_multigpu:True_labels:ds_frac:1_TEST.csv', sep='\t', header=0)
test45['Sentence'] = test45['Sentence'].str.lower()
test45_personal = test45[test45['Sentence'].str.contains(' me ') | test45['Sentence'].str.contains(' my ') | test45['Sentence'].str.contains(' we ')]

test78 = pd.read_csv('../../preds/test/classifier_askparents_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:78_multigpu:True_labels:ds_frac:1_TEST.csv', sep='\t', header=0)
test78['Sentence'] = test78['Sentence'].str.lower()
test78_personal = test78[test78['Sentence'].str.contains(' me ') | test78['Sentence'].str.contains(' my ') | test78['Sentence'].str.contains(' we ')]

In [58]:
print(classification_report(test2_personal['DS_Label'], test2_personal['Pred_Label'], digits=4))

              precision    recall  f1-score   support

           0     0.8408    0.9429    0.8889       140
           1     0.3333    0.1379    0.1951        29

    accuracy                         0.8047       169
   macro avg     0.5870    0.5404    0.5420       169
weighted avg     0.7537    0.8047    0.7698       169



In [59]:
print(classification_report(test13_personal['DS_Label'], test13_personal['Pred_Label'], digits=4))

              precision    recall  f1-score   support

           0     0.8741    0.8929    0.8834       140
           1     0.4231    0.3793    0.4000        29

    accuracy                         0.8047       169
   macro avg     0.6486    0.6361    0.6417       169
weighted avg     0.7967    0.8047    0.8004       169



In [60]:
print(classification_report(test20_personal['DS_Label'], test20_personal['Pred_Label'], digits=4))

              precision    recall  f1-score   support

           0     0.8581    0.9071    0.8819       140
           1     0.3810    0.2759    0.3200        29

    accuracy                         0.7988       169
   macro avg     0.6195    0.5915    0.6010       169
weighted avg     0.7762    0.7988    0.7855       169



In [61]:
print(classification_report(test45_personal['DS_Label'], test45_personal['Pred_Label'], digits=4))

              precision    recall  f1-score   support

           0     0.8609    0.9286    0.8935       140
           1     0.4444    0.2759    0.3404        29

    accuracy                         0.8166       169
   macro avg     0.6527    0.6022    0.6169       169
weighted avg     0.7895    0.8166    0.7986       169



In [62]:
print(classification_report(test78_personal['DS_Label'], test78_personal['Pred_Label'], digits=4))

              precision    recall  f1-score   support

           0     0.8627    0.9429    0.9010       140
           1     0.5000    0.2759    0.3556        29

    accuracy                         0.8284       169
   macro avg     0.6814    0.6094    0.6283       169
weighted avg     0.8005    0.8284    0.8074       169



In [53]:
dev_personal = dev[dev['Sentence'].str.contains(' i ') | dev['Sentence'].str.contains(' I ')]
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
dev_personal[dev_personal['DS_Label']==1]

,Majority_label,DS_Label,Sentence,Post.ID,Reply.ID,Sent.Num,Pred_Label,Question
ID,,,,,,,,
5l0ln3-2.1-0,0,1,Yup this is why I went straight to suggesting white noise etc .,5l0ln3,2.1,0,0,"Maybe bed time is none of my business But, how can I approach the parents and get them to quiet their kid at night How to confront neighbors about their lack of control"
71r6gd-2-0,1,1,If you 're not a vegan I would explain how you get all your food .,71r6gd,2.0,0,1,[deleted] How to explain to a kid the reality of going hunting
4pn5x8-5-2,1,1,Would I do it ?,4pn5x8,5.0,2,0,"I witnessed this today and just looking for other parents opinion/experience on it What age do you think it is appropriate to leave a child in a car not running with the windows down in a private daycare parking lot, the daycare is locked to people coming in and you have no visual of the car or child while you are in the building Age of child left in car"
dgpwtd-2-1,1,1,"Unless it 's a safety issue , I do n't put my hands on my kids in a moment like that - it sends the message that I 'm allowed to touch their body without consent .",dgpwtd,2.0,1,0,Where to draw the line What is reasonable in that situation Should I have picked him up and sat him in the next room like his mom did How much force is reasonable to use on a 4 y/o?
c823ha-4-0,1,1,Could be teething but if he 's like that more than a few days I would take him to the doctor .,c823ha,4.0,0,0,"I have a ten month old who is a good baby and hardly ever cry’s but ere lately it’s all he’s done I’ve tried everything I’m about to lose my sanity if this doesn’t stop, what can I do Why will my 10 month old not stop crying?"
54y27a-2-2,0,1,I had a therapist and psychiatrist I saw regularly during and after my son was born .,54y27a,2.0,2,0,Any tips or tricks Are you glad you decided to become a parent Thank you Parenting with a History of Depression?
7kxtia-2-0,0,1,"Spanking where I live is illegal , full stop .",7kxtia,2.0,0,0,"However,I am astounded to the amount of fellow college students who admit their parents still hit them for misbehaving I personally am still hit by my dad for doing something that “annoys” him What is your stance on spanking/hitting older children How old is too old to use corporal punishment on children?"
dgpwtd-2-3,1,1,"I 'd say , "" I wo n't let you hurt me .",dgpwtd,2.0,3,1,Where to draw the line What is reasonable in that situation Should I have picked him up and sat him in the next room like his mom did How much force is reasonable to use on a 4 y/o?
6e5o54-1-0,1,1,Toys are one of the things I am ( a ) least likely to buy online and ( b ) least likely to rely on reviews for .,6e5o54,1.0,0,1,Do you read reviews before buying a toy?


In [46]:
dev_personal.shape

(111, 8)

To verify if looking up sentences with 'me', 'my' and 'we' has more instances of personal narrative, we manually looked at the sentences in the dev set. I counted at least 64 sentences which are plausibly personal narrative

In [54]:
dev['Sentence'] = dev['Sentence'].str.lower()
dev_personal = dev[dev['Sentence'].str.contains(' we ') | dev['Sentence'].str.contains(' me ') | dev['Sentence'].str.contains(' my ')]

dev_personal

,Majority_label,DS_Label,Sentence,Post.ID,Reply.ID,Sent.Num,Pred_Label,Question
ID,,,,,,,,
bizagh-4-1,0,0,did n’t screw me up any except maybe having a sense of humor that was a little too advanced for my age,bizagh,4.0,1,0,I feel like I need to do damage control Her grandparents think I’m over reacting but I honestly don’t give a shit I feel like I failed as a parent So my three year old watched something awful!
droq6l-1-3,0,0,"i keep 2 of each size diaper and a few wipes , as well as a compact water bottle to share , in my purse .",droq6l,1.0,3,1,What's in your diaper bag?
844azw-4-0,0,0,seems to me like a manipulation that got out of hand .,844azw,4.0,0,0,"I've actually thought maybe we should giver her MORE water before bed as that might satisfy her mental need to go She does not do this during the day at all, is not having accidents, and will use the potty by herself in any other situation It's only naps/nighttime Toddler saying she needs to go potty everytime she gets tucked into bed"
71r6gd-2-1,0,0,the whole circle of life and we 're the top of the food chain .,71r6gd,2.0,1,0,[deleted] How to explain to a kid the reality of going hunting
ackme2-3-2,0,0,it 's not for me .,ackme2,3.0,2,0,Or am I in the wrong here Thanks a heap mums and dads <3 Feeling overly protected. Am I in the wrong here?
duvs75-1-0,1,1,parks and free building toys like lego or wooden blocks have worked for my kids .,duvs75,1.0,0,1,I imagine there are plenty of weekends with no major timesink activities Like what do you do with a toddler for that long Edit: assuming there is only 1kid and no siblings What exactly do you do with a toddler all weekend?
be9axi-4-1,0,0,"different families do this differently , we as adults and parents and people do not enjoy violent , aggressive , or overtly sexual content , so typically do n't consume it , its not on a whole lot .",be9axi,4.0,1,0,TL DR: What’s your rule for TV content Why do you do it this way Violent TV shows and dramas
be9axi-3-0,1,1,"i let my kids watch most things , pg and below .",be9axi,3.0,0,1,TL DR: What’s your rule for TV content Why do you do it this way Violent TV shows and dramas
dp6rdz-3.1-0,0,0,our next door neighbors gave my son mini m&ms at halloween when he was 1 .,dp6rdz,3.1,0,0,"I don’t have kids, I honestly don’t know much about them But, a neighbour across the street has a really sweet one year old that they will be bringing around just to the neighbors houses for Halloween Is there anything I could give him that he could eat at this age (not just a banana or some other fruit) Anything a 1 year old can eat on Halloween?"


In [ ]:
12345678910111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364

In [55]:
dev_personal.shape

(109, 8)

In [56]:
print(classification_report(dev_personal['DS_Label'], dev_personal['Pred_Label']))

              precision    recall  f1-score   support

           0       0.76      0.79      0.77        78
           1       0.41      0.35      0.38        31

    accuracy                           0.67       109
   macro avg       0.58      0.57      0.58       109
weighted avg       0.66      0.67      0.66       109



In [65]:
dev_personal['Sentence'].tolist()

['did n’t screw me up any except maybe having a sense of humor that was a little too advanced for my age',
 'i keep 2 of each size diaper and a few wipes , as well as a compact water bottle to share , in my purse .',
 'seems to me like a manipulation that got out of hand .',
 "the whole circle of life and we 're the top of the food chain .",
 "it 's not for me .",
 'parks and free building toys like lego or wooden blocks have worked for my kids .',
 "different families do this differently , we as adults and parents and people do not enjoy violent , aggressive , or overtly sexual content , so typically do n't consume it , its not on a whole lot .",
 'i let my kids watch most things , pg and below .',
 'our next door neighbors gave my son mini m&ms at halloween when he was 1 .',
 'a costco employee tried to give my 2 year old a pork rind ( free sample stand ) and when my partner explained that he \'s a vegetarian , she tried to hand it directly to my son and said , " well we just wo n\'t